### Topic 1. Not all VEP-representative isoforms are in the MSKCC and UniProt isoform list

<font color="red">Terminology alert!</font>
- MSK isoforms: https://github.com/mskcc/vcf2maf/blob/main/data/isoform_overrides_at_mskcc
- UniProt isoforms: https://github.com/mskcc/vcf2maf/blob/main/data/isoform_overrides_uniprot

Among many isoforms that encompasses a variant, VEP selects a representative transcript writes only its information in its 'Transcript_ID', 'HGVSc', 'HGVSp', etc. columns
It's said,

<i>"To identify this transcript, we consider, where available, evidence of functional potential (such as evolutionary conservation of a coding region, transcript expression levels), transcript length and evidence from other resources (such as concordance with the APPRIS1 ’principal isoform’ and with the UniProt/Swiss-Prot ‘canonical isoform - functional, expressed, conserved, has consensus; if not choose longest’)."</i> [link: http://mart.ensembl.org/info/genome/genebuild/canonical.html]

Since I was told that running `MafAnnotator.py` using its protein querying option emits errors, probably due to the input variants not labeled with isoforms matching that of OncoKB, I tried to find the proportion of isoforms matching that of OncoKB (isoforms from either 'MSK isoforms' or 'UniProt isoforms'). Since I knew VEP maf files also included an 'all_effects' column that includes information for all other VEP isoforms, I might probably be able to re-match these isoforms to that of OncoKB.

In [1]:
import pandas as pd

In [2]:
project = "18328"

In [28]:
flt = pd.read_table(f"{project}/results/cohort_filtered.maf") # here, the project directory has been soft-linked to this directory
flt.head()

/tmp/ipykernel_29513/2203655544.py:1: DtypeWarning: Columns (0,5,6,113,114,145,156) have mixed types. Specify dtype option on import or set low_memory=False.
  flt = pd.read_table(f"{project}/results/cohort_filtered.maf") # here, the project directory has been soft-linked to this directory


,genomic_vcf37_source,chr_hg19,pos_hg19,ref_hg19,alt_hg19,clinical_significance_brcaexchange,brcaexchange,Hugo_Symbol,Entrez_Gene_Id,Center,...,HIGHEST_LEVEL,CITATIONS,LEVEL_Dx1,LEVEL_Dx2,LEVEL_Dx3,HIGHEST_DX_LEVEL,LEVEL_Px1,LEVEL_Px2,LEVEL_Px3,HIGHEST_PX_LEVEL
0,NaN,17,7577094,G,A,NaN,NaN,TP53,0,.,...,NaN,21445056,NaN,NaN,NaN,NaN,"AMLMRC,AML,TMN,CLLSLL,MDS,ET,MPN,PMF",NaN,MCL,LEVEL_Px1
1,NaN,17,7578190,T,C,NaN,NaN,TP53,0,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,12,56477631,T,A,NaN,NaN,ERBB3,0,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,17,7574003,G,A,NaN,NaN,TP53,0,.,...,NaN,19336573;27759562;16007150;11753428;11900253;2...,NaN,NaN,NaN,NaN,"AMLMRC,AML,TMN,CLLSLL,MDS,ET,MPN,PMF",NaN,MCL,LEVEL_Px1
4,NaN,1,115256530,G,T,NaN,NaN,NRAS,0,.,...,LEVEL_3A,10574788;9219684;20194776;23515407;26619011;29...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### - MSKCC + Uniprot isoforms 

In [4]:
isoform_mskcc = pd.read_table("isoform_overrides_at_mskcc.txt")
isoform_mskcc.head()

,#enst_id,gene_name,refseq_id
0,ENST00000318560,ABL1,NM_005157.4
1,ENST00000321945,ABRAXAS1,NM_139076.2
2,ENST00000434821,ACVR1,NM_001111067.2
3,ENST00000373204,AGO1,NM_012199.2
4,ENST00000220592,AGO2,NM_012154.3


In [5]:
isoform_uniprot = pd.read_table("isoform_overrides_uniprot.txt")
isoform_uniprot.head()

,#enst_id,gene_name,refseq_id,ccds_id
0,ENST00000263100,A1BG,NM_130786.3,CCDS12976.1
1,ENST00000373993,A1CF,NM_138932.2,CCDS7242.1
2,ENST00000318602,A2M,NM_000014.4,CCDS44827.1
3,ENST00000299698,A2ML1,NaN,CCDS8596.2
4,ENST00000249005,A4GALT,NM_017436.4,CCDS14041.1


#### - Compare Isabl generate_maf result isoforms with MSKCC isoforms

In [10]:
mskcc_set = set(isoform_mskcc['#enst_id']) # Isoforms in MSKCC
uniprot_set = set(isoform_uniprot['#enst_id']) # Isoforms in UniProt
both_set = mskcc_set | uniprot_set # Union (~= both)

In [11]:
def parse_all_effects(row, both_set):
    """Checks if there is another isoform name within all_effects column
    that is included in both sets (MSKCC + UniProt)"""
    assert hasattr(row, "all_effects"), f"ERROR: row does not have all_effects:\n{row}"
    isoforms = row.all_effects.split(';')
    for isoform in isoforms:
        if len(isoform) < 1: continue # skip empty values
        if isoform.count(',') >= 4:
            features = isoform.split(',', 4)
            gene_sym, effect, protein_change, enst_id, nm_id = features
        else:
            continue
        if enst_id in both_set:
            return (enst_id, protein_change, effect)
        else:
            continue
    return ("n/a", "n/a", "n/a")

In [12]:
%%time
def get_excluded_genes(df):
    """Checks is a representative VEP isoform is in both_set, but if not, 
    checks again if another isoform from the all_effects column is in both_set,
    and if there is such isoform, labels the row 'spared', and if not,
    labels the row 'excluded', and returns a list of df indices that are excluded"""
    included = spared = excluded = 0
    genes_spared = set()
    genes_excluded = set()
    idx_excluded = list()
    excluded_df = pd.DataFrame(columns = df.columns)
    for ix, row in enumerate(df.iterrows()):
        row = row[1] # ignore index column
        if row.Transcript_ID in both_set: # if ENST transcript id in MSKCC+UniProt database
            included += 1
        else:
            enst_id, protein_change, effect = parse_all_effects(row, both_set) #
            if enst_id == 'n/a':
                excluded += 1
                genes_excluded.add(row.Transcript_ID)
                idx_excluded.append(ix)
            else:
                included += 1
                spared += 1
                genes_spared.add(enst_id)

    print(f'included: {included} (spared: {spared}), excluded: {excluded}') # debug
    return idx_excluded

#raw_idx_excluded = get_excluded_genes(raw)
flt_idx_excluded = get_excluded_genes(flt)

included: 10833 (spared: 2147), excluded: 650
CPU times: user 689 ms, sys: 17.4 ms, total: 707 ms
Wall time: 726 ms


##### - Ratio of included (in MSKCC/UniProt), not included but spared, excluded

In [23]:
ratio_included = (10833-2147)/flt.shape[0]
ratio_spared = 2147/flt.shape[0]
ratio_spared_plus_included = 10833/flt.shape[0]
ratio_excluded = 650/flt.shape[0]

print(f"""included (before sparing): {ratio_included:.2f}, 
spared: {ratio_spared:.2f},
included + spared: {ratio_spared_plus_included:.2f},
excluded: {ratio_excluded:.2f}""")

included (before sparing): 0.76, 
spared: 0.19,
included + spared: 0.94,
excluded: 0.06


- You can see that 76% isoforms from cohort_filtered maf is not in (MSKCC or UniProt) isoforms
- Still, 19% of them still have matching isoforms in their 'all_effects' column
- But 6% of them cannot be matched even trying to find a match in the 'all_effects' column

##### - Counts of Variant_Classification (just to see what types of variants we're not matching/excluding, and matching/including)

In [69]:
# isoforms not-included in our databases
print(flt.iloc[flt_idx_excluded].Variant_Classification.value_counts()) 
print(flt.iloc[flt_idx_excluded].MUTATION_EFFECT.value_counts())
print(flt.iloc[flt_idx_excluded].ONCOGENIC.value_counts())

Splice_Region      403
Splice_Site        245
Frame_Shift_Del      2
Name: Variant_Classification, dtype: int64
Likely Loss-of-function    650
Name: MUTATION_EFFECT, dtype: int64
Likely Oncogenic    650
Name: ONCOGENIC, dtype: int64


In [70]:
# included (including spared) in our databases
print(flt.drop(flt_idx_excluded).Variant_Classification.value_counts())
print(flt.drop(flt_idx_excluded).MUTATION_EFFECT.value_counts())
print(flt.drop(flt_idx_excluded).ONCOGENIC.value_counts())

Splice_Region             8493
Frame_Shift_Ins            544
Nonsense_Mutation          424
Nonstop_Mutation           364
Missense_Mutation          340
Splice_Site                261
Frame_Shift_Del            222
In_Frame_Ins               151
Translation_Start_Site      23
In_Frame_Del                11
Name: Variant_Classification, dtype: int64
Likely Loss-of-function    10371
Unknown                      379
Loss-of-function              44
Likely Gain-of-function       18
Inconclusive                  12
Gain-of-function               9
Name: MUTATION_EFFECT, dtype: int64
Likely Oncogenic       10411
Predicted Oncogenic      229
Oncogenic                 31
Inconclusive              12
Name: ONCOGENIC, dtype: int64


##### Save as CSVs: genes not in OncoKB, and genes in OncoKB - just so for whatever twisted reasons I could access them later

In [26]:
flt.iloc[flt_idx_excluded].to_csv(f"{project}.cohort_filtered.genes_not_in_oncokb.maf.csv")
flt.drop(flt_idx_excluded).to_csv(f"{project}.cohort_filtered.genes_in_oncokb.maf.csv")

###

### Topic 2. Annotation differences between: protein-queried results, and genomic position (default)-queried results

While doing this isoforms study I found out that `MafAnnotator.py` results differ (even with the same input) when queried with genomic positions (g-queries) or protein change (p-queries).
<br>Actually
- p-queries fail to annotate some missense mutations and in-frame deletions, while
- q-queries usually fail to annotate some splice site mutations

##### MafAnnotator output for genomic queries

In [36]:
g_annot = pd.read_table("./query_comparison/merged_filtered.g_annotated.maf")
g_annot.to_csv("./query_comparison/merged_filtered.g_annotated.maf.csv")
g_annot.head()

/tmp/ipykernel_29513/1478954808.py:1: DtypeWarning: Columns (1,2,5,6,8,11,14,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,57,58,59,60,61,62,65,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,110,111,112,113,114,115,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,140,142,143,144,145,146,147,156,161,163,164,165,167,168,171,172,173,174,175,176,177,178,179,180,181,182) have mixed types. Specify dtype option on import or set low_memory=False.
  g_annot = pd.read_table("./query_comparison/merged_filtered.g_annotated.maf")


,genomic_vcf37_source,chr_hg19,pos_hg19,ref_hg19,alt_hg19,clinical_significance_brcaexchange,brcaexchange,Hugo_Symbol,Entrez_Gene_Id,Center,...,LEVEL_Dx1.1,LEVEL_Dx2.1,LEVEL_Dx3.1,HIGHEST_DX_LEVEL.1,DX_CITATIONS,LEVEL_Px1.1,LEVEL_Px2.1,LEVEL_Px3.1,HIGHEST_PX_LEVEL.1,PX_CITATIONS
0,NaN,17,7577094,G,A,NaN,NaN,TP53,0,.,...,NaN,NaN,NaN,NaN,NaN,"AMLMRC,AML,TMN,CLLSLL,MDS,ET,MPN,PMF",NaN,MCL,LEVEL_Px1,25412851;25860933;22186996;18596741;25412846;2...
1,NaN,17,7578190,T,C,NaN,NaN,TP53,0,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,12,56477631,T,A,NaN,NaN,ERBB3,0,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,17,7574003,G,A,NaN,NaN,TP53,0,.,...,NaN,NaN,NaN,NaN,NaN,"AMLMRC,AML,TMN,CLLSLL,MDS,ET,MPN,PMF",NaN,MCL,LEVEL_Px1,25412851;25860933;22186996;18596741;25412846;2...
4,NaN,1,115256530,G,T,NaN,NaN,NRAS,0,.,...,NaN,"CMML,JMML,MDS",ETPLL,LEVEL_Dx2,"26341525;23690417;Patel, B. et al., Abstract# ...",MDS,NaN,NaN,LEVEL_Px1,24881041;21714648;8329714;24220272


##### MafAnnotator output for protein queries

In [38]:
p_annot = pd.read_table("./query_comparison/merged_filtered.p_annotated.maf")
p_annot.to_csv("./query_comparison/merged_filtered.p_annotated.maf.csv")
p_annot.head()

/tmp/ipykernel_29513/3966712830.py:1: DtypeWarning: Columns (1,2,5,6,8,11,14,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,57,58,59,60,61,62,65,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,110,111,112,113,114,115,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,140,142,143,144,145,146,147,156,161,163,164,165,167,168,171,172,173,174,175,176,177,178,179,180,181,182) have mixed types. Specify dtype option on import or set low_memory=False.
  p_annot = pd.read_table("./query_comparison/merged_filtered.p_annotated.maf")


,genomic_vcf37_source,chr_hg19,pos_hg19,ref_hg19,alt_hg19,clinical_significance_brcaexchange,brcaexchange,Hugo_Symbol,Entrez_Gene_Id,Center,...,LEVEL_Dx1.1,LEVEL_Dx2.1,LEVEL_Dx3.1,HIGHEST_DX_LEVEL.1,DX_CITATIONS,LEVEL_Px1.1,LEVEL_Px2.1,LEVEL_Px3.1,HIGHEST_PX_LEVEL.1,PX_CITATIONS
0,NaN,17,7577094,G,A,NaN,NaN,TP53,0,.,...,NaN,NaN,NaN,NaN,NaN,"AMLMRC,AML,TMN,CLLSLL,MDS,ET,MPN,PMF",NaN,MCL,LEVEL_Px1,25412851;25860933;22186996;18596741;25412846;2...
1,NaN,17,7578190,T,C,NaN,NaN,TP53,0,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,12,56477631,T,A,NaN,NaN,ERBB3,0,.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,17,7574003,G,A,NaN,NaN,TP53,0,.,...,NaN,NaN,NaN,NaN,NaN,"AMLMRC,AML,TMN,CLLSLL,MDS,ET,MPN,PMF",NaN,MCL,LEVEL_Px1,25412851;25860933;22186996;18596741;25412846;2...
4,NaN,1,115256530,G,T,NaN,NaN,NRAS,0,.,...,NaN,"CMML,JMML,MDS",ETPLL,LEVEL_Dx2,"26341525;23690417;Patel, B. et al., Abstract# ...",MDS,NaN,NaN,LEVEL_Px1,24881041;21714648;8329714;24220272


My apologies! `'VARIANT_IN_ONCOKB'` column was made by MafAnnotator when the ran with the default options (genomic queryig),
<br>while `'VARIANT_IN_ONKOKB.1'` column was added by MafAnnotator when I used the MafAnnotator output again as input, using the protein querying option

In [44]:
# Variants with: protein-queried annotations in OncoKB & genomic-queried annotations NOT in OncoKB
pO_gX = p_annot[(p_annot['VARIANT_IN_ONCOKB'] != True) & (p_annot['VARIANT_IN_ONCOKB.1'] == True)]
pO_gX.to_csv("merged_filtered.g-X_p-O.maf.csv")
pO_gX['Consequence'].value_counts()

missense_variant    17
inframe_deletion    13
Name: Consequence, dtype: int64

In [46]:
# Variants with: protein-queried annotations NOT in OncoKB & genomic-queried annotations in OncoKB
pX_gO = p_annot[(p_annot['VARIANT_IN_ONCOKB'] == True) & (p_annot['VARIANT_IN_ONCOKB.1'] != True)]
pX_gO.to_csv("merged_filtered.g-O_p-X.maf.csv")
pX_gO['Consequence'].value_counts()

missense_variant                                                  13
splice_donor_variant                                              13
splice_acceptor_variant                                            9
splice_donor_variant,frameshift_variant                            9
frameshift_variant                                                 9
splice_acceptor_variant,coding_sequence_variant,intron_variant     5
stop_gained                                                        4
splice_acceptor_variant,coding_sequence_variant                    1
splice_donor_variant,missense_variant                              1
splice_donor_variant,intron_variant                                1
Name: Consequence, dtype: int64

You can see that MafAnnotator results differ by the querying option of MafAnnotator.py
<br>Interestingly, the Mutation consequence differ by the querying option itself!